In [1]:
import pandas as pd
import requests
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

In [101]:
def get_pdp(URL):
    try:
        f = open("keys.txt", "r")
        r_keys = f.readlines()
        keys =[]
        for i in r_keys:
            keys.append(i[:-1])
        if URL.count("google",0,35) or URL.count("yahoo",0,35) or URL.count("bing",0,35) or URL.count("youtube",0,35) or URL.count("facebook",0,35) or URL.count("wikipedia",0,35)> 0:
            class_pdp = "search_page"
        elif len(URL) < 35:
            class_pdp = "skipped"
        elif all(x in URL for x in ['amazon','/gp/aw/d/']) or \
            all(x in URL for x in ['amazon','/dp/']) or \
            all(x in URL for x in ['target','/p/']) or \
            all(x in URL for x in ['walmart','/ip/']) or \
            all(x in URL for x in ['walmart','track']):
            class_pdp = "YES"
        elif all(x in URL for x in ['amazon','/stores/']) or \
            all(x in URL for x in ['target','/b/']) or \
            all(x in URL for x in ['target','/c/']) or \
            all(x in URL for x in ['walmart','browse']) or \
            all(x in URL for x in ['walmart','/cp']) or \
            all(x in URL for x in ['target','searchTerm']) or \
            all(x in URL for x in ['walmart','search']) or \
            all(x in URL for x in ['amazon','/goldbox']) or \
            all(x in URL for x in ['amazon','/offers']) or \
            all(x in URL for x in ['amazon','/angebot']) or \
            all(x in URL for x in ['target','/top-deals']) or \
            all(x in URL for x in ['target','/circle']) or \
            all(x in URL for x in ['walmart','/m']) or \
            all(x in URL for x in ['walmart','/deals']) or \
            all(x in URL for x in ['amazon','Best-Sellers']) or \
            all(x in URL for x in ['amazon','/ask','/questions']) or \
            all(x in URL for x in ['amazon','/product-reviews/']) or \
            all(x in URL for x in ['amazon','/c/ref=mw_dp_buy_crt']) or \
            all(x in URL for x in ['amazon','/cart/']) or \
            all(x in URL for x in ['amazon','/huc/view.html?']) or \
            all(x in URL for x in ['target','/co-cart']) or \
            all(x in URL for x in ['target','/cart']) or \
            all(x in URL for x in ['amazon','bestsellers','Best-Sellers']):
            class_pdp = "Other_pages"
        else:
            HEADERS = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64)AppleWebKit/537.36 (KHTML, like Gecko)Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})
            webpage = requests.get(URL, headers=HEADERS, timeout=25)
            soup = BeautifulSoup(webpage.content, "html5lib")
            text = soup.find_all(text=True)
            if text == None:
                class_pdp = "No Data"
            else:
                new = []
                for i in text:
                    new.append(i.lower())
                    mystr =" ".join(new)
                pdp = [i for i in new if i in keys]
                if len(pdp) > 0 and len(pdp)< 10:
                    class_pdp = "YES"
                else:
                    class_pdp = "NO"
    except Exception as e:
        print(e)
        class_pdp = "error or timeout"
    return class_pdp

In [111]:
df = pd.read_csv('data1859.csv')
url_list = df['URL'].to_list()
urls = url_list[:10871]

In [ ]:
for URL in urls:
    pdp_tag = get_pdp(URL)
    f = open("pdp_tags_2.txt", "a+")
    f.write(pdp_tag + "\n")
    f.close()

In [112]:
f = open("pdp_tags_2.txt", "r")
pdp_tag = f.readlines()

pdp =[]
for i in pdp_tag:
    pdp.append(i[:-1])

len(pdp)

4593

In [113]:
df['PDP_tag'] = pdp
df.to_csv('data1859_ped.csv', index=False)

In [106]:
new_df

,Path Analysis,PDP Flag,PDP_tag
0,https://www.target.com/,0.0,skipped
1,https://www.target.com/s?searchTerm=hair+removal,0.0,Other_pages
2,https://www.target.com/p/spa-sciences-sima-son...,1.0,YES
3,https://www.amazon.com/s?crid=1TL9445LWNRYU&k=...,0.0,NO
4,https://www.amazon.com/Philips-Norelco-Shaver-...,1.0,YES
...,...,...,...
7030,https://www.amazon.com/ap/signin?openid.assoc_...,0.0,NO
7031,https://www.amazon.com/Finishing-Touch-Flawles...,1.0,YES
7032,https://www.amazon.com/Finishing-Touch-Flawles...,1.0,YES
7033,https://www.amazon.com/Finishing-Touch-Flawles...,1.0,YES
